In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cspes\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() 
intents = json.loads(data_file)

In [13]:
intents['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'context_set': ''},
 {'tag': 'creator',
  'patterns': ['what is the name of your developers',
   'what is the name of your creators',
   'what is the name of the developers',
   'what is the name of the creators',
   'who created you',
   'your developers',
   'your creators',
   'who are your developers',
   'developers',
   'you are made by',
   'you are made by whom',
   'who

In [14]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        # combination between patterns and intents
        documents.append((w, intent['tag']))# add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [15]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cspes\AppData\Roaming\nltk_data...


True

In [16]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Cspes\AppData\Roaming\nltk_data...


True

In [17]:
# 'lemmatize' reduce each word to root and remove duplicates ex. [Changing, Changes, Changed] -> Change || [Studying, Studies] -> Study
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

# Modeling

In [18]:
# training
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [19]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [20]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[-0.08948738 -0.03311509  0.04218775 ... -0.00596585 -0.02917154
   0.04261629]
 [ 0.11625303  0.06327084  0.0863461  ...  0.00612181  0.0814839
   0.10212947]
 [ 0.0099726   0.10903893  0.06341523 ...  0.0616308   0.0665537
   0.026885  ]
 ...
 [-0.02311346  0.01947165 -0.0291133  ...  0.02302411  0.06177375
  -0.00817733]
 [-0.12309909  0.04137966 -0.1123822  ... -0.0829576  -0.01478203
   0.10969703]
 [-0.00691538 -0.11018842 -0.03948733 ...  0.02546465 -0.01948723
   0.0219527 ]]


In [21]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
81/81 [==============================] - 1s 1ms/step - loss: 3.6289 - accuracy: 0.0519
Epoch 2/200
81/81 [==============================] - 0s 1ms/step - loss: 3.4919 - accuracy: 0.1506
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.2980 - accuracy: 0.1926
Epoch 4/200
81/81 [==============================] - 0s 1ms/step - loss: 3.0166 - accuracy: 0.2272
Epoch 5/200
81/81 [==============================] - 0s 1ms/step - loss: 2.7358 - accuracy: 0.3086
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 2.4618 - accuracy: 0.3827
Epoch 7/200
81/81 [==============================] - 0s 2ms/step - loss: 2.1876 - accuracy: 0.4444
Epoch 8/200
81/81 [==============================] - 0s 1ms/step - loss: 2.0399 - accuracy: 0.4790
Epoch 9/200
81/81 [==============================] - 0s 1ms/step - loss: 1.8053 - accuracy: 0.5556
Epoch 10/200
81/81 [==============================] - 0s 1ms/step - loss: 1.6449 - accuracy: 0.5877
Epoch 11/